In [37]:
import os
import pandas as pd
import statistics
import itertools


In [38]:
# Load the dataframe from the CSV file

df = pd.read_csv(os.path.abspath('Experimental Data/equiatomic_data.csv'))

In [39]:
# Filter out elements not in CANTOR compositions

elements = ['Ni', 'Co', 'Fe', 'Mn', 'Cr']


CANTOR_elements_df = df[df['FORMULA'].str.findall('[A-Z][a-z]*').apply(lambda x: all(element in elements for element in x))]

In [40]:
# Create a new column with the formulas in alphabetical order this is used to deal with duplicates e.g FeCoCr = CrFeCo

CANTOR_elements_df.loc[:, 'Alphabetical Formula'] = CANTOR_elements_df['FORMULA'].apply(lambda f: ''.join(sorted(f)))

/tmp/ipykernel_70/1747206503.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CANTOR_elements_df.loc[:, 'Alphabetical Formula'] = CANTOR_elements_df['FORMULA'].apply(lambda f: ''.join(sorted(f)))


In [41]:
# Keep only T = 298K

CANTOR_RT_df = pd.DataFrame() 

for index, row in CANTOR_elements_df.iterrows():
    if row["Test temperature (K)"] == 298:
        CANTOR_RT_df = CANTOR_RT_df.append(row, ignore_index=True)

/tmp/ipykernel_70/272123220.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CANTOR_RT_df = CANTOR_RT_df.append(row, ignore_index=True)
/tmp/ipykernel_70/272123220.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CANTOR_RT_df = CANTOR_RT_df.append(row, ignore_index=True)
/tmp/ipykernel_70/272123220.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CANTOR_RT_df = CANTOR_RT_df.append(row, ignore_index=True)
/tmp/ipykernel_70/272123220.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CANTOR_RT_df = CANTOR_RT_df.append(row, ignore_index=True)
/tmp/ipykernel_70/272123220.py:7: FutureWarning: The frame.append method is deprecated a

/tmp/ipykernel_70/272123220.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CANTOR_RT_df = CANTOR_RT_df.append(row, ignore_index=True)
/tmp/ipykernel_70/272123220.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CANTOR_RT_df = CANTOR_RT_df.append(row, ignore_index=True)
/tmp/ipykernel_70/272123220.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CANTOR_RT_df = CANTOR_RT_df.append(row, ignore_index=True)
/tmp/ipykernel_70/272123220.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CANTOR_RT_df = CANTOR_RT_df.append(row, ignore_index=True)
/tmp/ipykernel_70/272123220.py:7: FutureWarning: The frame.append method is deprecated a

In [42]:
# Find all unique compositions using alphabetically sorted formula

unique_comp_data = {}

for composition in CANTOR_RT_df['Alphabetical Formula'].unique().tolist():
    unique_comp_data[composition] = []

In [43]:
# Append experimenatl yield stress data to unique compoistions

for index, row in CANTOR_RT_df.iterrows():
    
    stress = row["Yield Stress (MPA)"]
    
    marker = row["Alphabetical Formula"]
    
    unique_comp_data[marker].append(stress)

In [44]:
# Take median points of experimental data to remove outliers

ave_dict = {}

for composition in unique_comp_data:
    ave_dict[composition] = []
    
    ave_stress = statistics.median(map(float, unique_comp_data[composition]))
    ave_dict[composition] = ave_stress
    
print(ave_dict)

{'CFNeio': 210.0, 'CCFNeior': 273.0, 'CCMNinor': 282.0, 'CCNior': 471.5, 'CFMNeino': 170.0, 'CMNino': 321.0, 'FMNein': 407.0, 'CCFMNeinor': 378.7}


In [45]:
# We now have median yield stress data (at 298K) for differenent equiatomic alloys comprised from CANTOR elements 
# To compare experimental values to theoretical we need to replace the alphabetically sorted formula with actual formula

In [46]:
# From elemental data list generate possible combinations

Elemental_data = {
    "Goldschmidt Radii": {'Ni':1.24, 'Co':1.25, 'Fe':1.26, 'Mn':1.27, 'Cr':1.28},
    "Atomic Volume": {'Ni':10.94, 'Co':11.12, 'Fe':12.09, 'Mn':12.60, 'Cr':12.27},
    "Shear Modulus": {'Ni':76, 'Co':76, 'Fe':82, 'Mn':44, 'Cr':115},
    "Poisson Ratio": { 'Ni':0.31, 'Co':0.31, 'Fe':0.28, 'Mn':0.26, 'Cr':0.21}
}

available_elements = list(Elemental_data["Goldschmidt Radii"].keys())

all_compoistions = []

for element in range(3, len(available_elements)+1):
    
        combinations = itertools.combinations(available_elements, element)
        
        for combo in combinations:
            all_compoistions.append("".join(combo))
        
print(all_compoistions)

['NiCoFe', 'NiCoMn', 'NiCoCr', 'NiFeMn', 'NiFeCr', 'NiMnCr', 'CoFeMn', 'CoFeCr', 'CoMnCr', 'FeMnCr', 'NiCoFeMn', 'NiCoFeCr', 'NiCoMnCr', 'NiFeMnCr', 'CoFeMnCr', 'NiCoFeMnCr']


In [47]:
# Create a dictionary where {Chemical Formula: Alphabetically sorted chemical formula} 
# This is required to rename experimental data values

alphabetical_key = {}   

for composition in all_compoistions:
    alphabetical_key[composition] = []
    
    alphabetical = sorted(list(composition))
    alphabetical_key[composition]  = "".join(alphabetical)
    
print(alphabetical_key)

{'NiCoFe': 'CFNeio', 'NiCoMn': 'CMNino', 'NiCoCr': 'CCNior', 'NiFeMn': 'FMNein', 'NiFeCr': 'CFNeir', 'NiMnCr': 'CMNinr', 'CoFeMn': 'CFMeno', 'CoFeCr': 'CCFeor', 'CoMnCr': 'CCMnor', 'FeMnCr': 'CFMenr', 'NiCoFeMn': 'CFMNeino', 'NiCoFeCr': 'CCFNeior', 'NiCoMnCr': 'CCMNinor', 'NiFeMnCr': 'CFMNeinr', 'CoFeMnCr': 'CCFMenor', 'NiCoFeMnCr': 'CCFMNeinor'}


In [48]:
# Replace alphabetically sorted formula names with chemical forumalas

experimental_data = {}

for k, v in ave_dict.items():
    for key, val in alphabetical_key.items():
        if val == k:
            experimental_data[key] = v
            break

print(experimental_data)

{'NiCoFe': 210.0, 'NiCoFeCr': 273.0, 'NiCoMnCr': 282.0, 'NiCoCr': 471.5, 'NiCoFeMn': 170.0, 'NiCoMn': 321.0, 'NiFeMn': 407.0, 'NiCoFeMnCr': 378.7}
